In [1]:
import faiss
import os
import numpy as np
import pandas as pd

import torch
from torch import Tensor
from torchvision import models

from torchvision.transforms import Compose, transforms
from PIL import Image
import cv2
import sqlite3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
df = pd.read_pickle('hnsw_all_image_results.pkl')

In [16]:
img_features = df['image_features']

img_features = np.array(img_features.to_list())
img_features = img_features.reshape(img_features.shape[0], img_features.shape[2])

In [17]:
img_features.shape

(10126, 2048)

In [18]:
# all image features will be in X
X = img_features

## Trying DBSCAN

In [113]:
from sklearn.cluster import DBSCAN

dbscan_cluster = DBSCAN(eps=3, min_samples=10).fit(X)

In [114]:
dbscan_cluster.labels_

array([ -1,  -1,   0, ..., 105, 105, 105])

In [115]:
max(dbscan_cluster.labels_)

105

In [116]:
df['cluster_id'] = dbscan_cluster.labels_

#### Accuracy

In [117]:
centroid_dataframe = pd.DataFrame(columns=["label", "num_correct", "num_total", "all_labels_list"])
for cluster_id in range(-1, max(dbscan_cluster.labels_)+1):
    # maximum image_dir
    predictions_for_centroid = df[df['cluster_id'] == cluster_id]
    value_counts = predictions_for_centroid['image_dir'].value_counts()
    
    labels = list()
    for row_id in value_counts.index:
        labels.append((os.path.split(row_id)[1], value_counts[row_id]))
    
    centroid_dataframe.loc[len(centroid_dataframe.index)] = [labels[0][0], labels[0][1], sum(value_counts), labels]

In [118]:
centroid_dataframe

,label,num_correct,num_total,all_labels_list
0,_Robert_Mugabe_falling_from_his_podium,80,8140,"[(_Robert_Mugabe_falling_from_his_podium, 80),..."
1,_This_cat_plotting_to_kill_someone,14,14,"[(_This_cat_plotting_to_kill_someone, 14)]"
2,_This_happy_family,21,21,"[(_This_happy_family, 21)]"
3,_Trump_trying_to_close_his_pen,36,36,"[(_Trump_trying_to_close_his_pen, 36)]"
4,_Dog_with_Cat_mask,10,10,"[(_Dog_with_Cat_mask, 10)]"
...,...,...,...,...
102,_Japanese_astronaut_looking_through_a_water_bu...,13,13,[(_Japanese_astronaut_looking_through_a_water_...
103,_A_New_Zealand_Lamb,10,10,"[(_A_New_Zealand_Lamb, 10)]"
104,_Ridiculously_photogenic_volleyball_player,14,14,"[(_Ridiculously_photogenic_volleyball_player, ..."
105,_A_guy_throwing_a_frisbee_really_hard,12,12,"[(_A_guy_throwing_a_frisbee_really_hard, 12)]"


## OPTICS

In [124]:
from sklearn.cluster import OPTICS

optic_cluster = OPTICS(n_jobs=50).fit(X)

In [125]:
optic_cluster.labels_

array([ -1,  -1, 178, ...,  66,  66,  66])

In [126]:
df['cluster_id'] = optic_cluster.labels_

In [127]:
max(optic_cluster.labels_)

226

In [128]:
centroid_dataframe = pd.DataFrame(columns=["label", "num_correct", "num_total", "all_labels_list"])
for cluster_id in range(-1, max(dbscan_cluster.labels_)+1):
    # maximum image_dir
    predictions_for_centroid = df[df['cluster_id'] == cluster_id]
    value_counts = predictions_for_centroid['image_dir'].value_counts()
    
    labels = list()
    for row_id in value_counts.index:
        labels.append((os.path.split(row_id)[1], value_counts[row_id]))
    
    centroid_dataframe.loc[len(centroid_dataframe.index)] = [labels[0][0], labels[0][1], sum(value_counts), labels]

In [129]:
centroid_dataframe

,label,num_correct,num_total,all_labels_list
0,_Peter_Dinklage_riding_a_scooter,73,7257,"[(_Peter_Dinklage_riding_a_scooter, 73), (_Rob..."
1,_Weightlifter_jumping_up_after_a_successful_lift,13,14,[(_Weightlifter_jumping_up_after_a_successful_...
2,_Headless_hockey_player_and_terrified_referee,8,8,[(_Headless_hockey_player_and_terrified_refere...
3,_This_teacher_from_my_Facebook_feed,5,5,"[(_This_teacher_from_my_Facebook_feed, 5)]"
4,_Obama_wielding_a_lightsaber_in_front_of_the_W...,13,14,[(_Obama_wielding_a_lightsaber_in_front_of_the...
...,...,...,...,...
102,_Albino_squirrel_leaning_on_a_screw,6,6,"[(_Albino_squirrel_leaning_on_a_screw, 6)]"
103,_A_guy_throwing_a_frisbee_really_hard,13,13,"[(_A_guy_throwing_a_frisbee_really_hard, 13)]"
104,_A_guy_throwing_a_frisbee_really_hard,8,8,"[(_A_guy_throwing_a_frisbee_really_hard, 8)]"
105,_2016_Olympic_Ping_Pong,13,13,"[(_2016_Olympic_Ping_Pong, 13)]"


## Trying Model training after PCA

In [130]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, random_state=22)
pca.fit(X)
pca_x = pca.transform(X)

In [133]:
pca_x.shape

(10126, 100)

In [132]:
# cluster training
ncentroids = 256
niter = 50
verbose = True
d = pca_x.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose, gpu=True)
kmeans.train(pca_x)

Clustering 10126 points in 100D to 256 clusters, redo 1 times, 50 iterations
  Preprocessing in 0.00 s


257574.46875

In [135]:
D, I = kmeans.index.search(pca_x, 1)
I.reshape(I.shape[0])
df['cluster_id'] = I

In [136]:
centroid_dataframe = pd.DataFrame(columns=["label", "num_correct", "num_total", "all_labels_list"])
for cluster_id in range(ncentroids):
    # maximum image_dir
    predictions_for_centroid = df[df['cluster_id'] == cluster_id]
    value_counts = predictions_for_centroid['image_dir'].value_counts()
    
    labels = list()
    for row_id in value_counts.index:
        labels.append((os.path.split(row_id)[1], value_counts[row_id]))
    
    centroid_dataframe.loc[len(centroid_dataframe.index)] = [labels[0][0], labels[0][1], sum(value_counts), labels]
    

In [138]:
sum(centroid_dataframe['num_correct'])/sum(centroid_dataframe['num_total'])

0.3296464546711436

## DBSCAN PCA

In [140]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, random_state=22)
pca.fit(X)
pca_x = pca.transform(X)
dbscan_cluster = DBSCAN(eps=3, min_samples=10).fit(pca_x)

In [141]:
max(dbscan_cluster.labels_)

114

In [142]:
df['cluster_id'] = dbscan_cluster.labels_

In [144]:
centroid_dataframe = pd.DataFrame(columns=["label", "num_correct", "num_total", "all_labels_list"])
for cluster_id in range(max(dbscan_cluster.labels_)):
    # maximum image_dir
    predictions_for_centroid = df[df['cluster_id'] == cluster_id]
    value_counts = predictions_for_centroid['image_dir'].value_counts()
    
    labels = list()
    for row_id in value_counts.index:
        labels.append((os.path.split(row_id)[1], value_counts[row_id]))
    
    centroid_dataframe.loc[len(centroid_dataframe.index)] = [labels[0][0], labels[0][1], sum(value_counts), labels]
    

In [148]:
centroid_dataframe

,label,num_correct,num_total,all_labels_list
0,_This_cat_plotting_to_kill_someone,14,14,"[(_This_cat_plotting_to_kill_someone, 14)]"
1,_This_happy_family,21,21,"[(_This_happy_family, 21)]"
2,_Trump_trying_to_close_his_pen,40,40,"[(_Trump_trying_to_close_his_pen, 40)]"
3,_Dog_with_Cat_mask,11,11,"[(_Dog_with_Cat_mask, 11)]"
4,_This_chubby_squirrel_(x-post_from__r_aww),10,10,"[(_This_chubby_squirrel_(x-post_from__r_aww), ..."
...,...,...,...,...
109,_Palestinian_man_taking_selfie_while_Israeli_p...,25,25,[(_Palestinian_man_taking_selfie_while_Israeli...
110,_Japanese_astronaut_looking_through_a_water_bu...,14,14,[(_Japanese_astronaut_looking_through_a_water_...
111,_A_New_Zealand_Lamb,11,11,"[(_A_New_Zealand_Lamb, 11)]"
112,_Ridiculously_photogenic_volleyball_player,16,16,"[(_Ridiculously_photogenic_volleyball_player, ..."


In [150]:
centroid_dataframe[centroid_dataframe['num_total']>50]

,label,num_correct,num_total,all_labels_list
69,_Canadian_PM_Justin_Trudeau_hesitating_to_shak...,57,57,[(_Canadian_PM_Justin_Trudeau_hesitating_to_sh...


In [147]:
sum(centroid_dataframe['num_total'])/114

19.964912280701753

In [146]:
sum(centroid_dataframe['num_correct'])/sum(centroid_dataframe['num_total'])

0.9912126537785588